In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
mainlog=pd.read_csv("MAMDAL_8_MAINLOG.csv")
mainlogtest=pd.read_csv("Mamdal3_WTLLIS_001_1.csv", low_memory = False)
hires=pd.read_csv("MAMDAL_8_HIRES.csv")
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils
from sklearn.cluster import KMeans

/home/poc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print(mainlogtest)

     wellName   datasetName      DEPT      AIBT      APOR   AVOL     BS  \
0         NaN           NaN         m      degC         %    ft3     mm   
1     Mamdal3  WTLLIS_001_1         2     -9999     -9999  -9999  215.9   
2     Mamdal3  WTLLIS_001_1       2.1     -9999     -9999  -9999  215.9   
3     Mamdal3  WTLLIS_001_1       2.2     -9999     -9999  -9999  215.9   
4     Mamdal3  WTLLIS_001_1       2.3     -9999     -9999  -9999  215.9   
5     Mamdal3  WTLLIS_001_1       2.4     -9999     -9999  -9999  215.9   
6     Mamdal3  WTLLIS_001_1       2.5     -9999     -9999  -9999  215.9   
7     Mamdal3  WTLLIS_001_1       2.6     -9999     -9999  -9999  215.9   
8     Mamdal3  WTLLIS_001_1       2.7     -9999     -9999  -9999  215.9   
9     Mamdal3  WTLLIS_001_1       2.8     -9999     -9999  -9999  215.9   
10    Mamdal3  WTLLIS_001_1       2.9     -9999     -9999  -9999  215.9   
11    Mamdal3  WTLLIS_001_1         3     -9999     -9999  -9999  215.9   
12    Mamdal3  WTLLIS_001

In [3]:
##mainlog=mainlog.drop(["MSPD","SPCG","GGCE","NPRL","DCOR","PDPE","HVOL","AVOL","TR21","TR11","TR22","TR12","BIT","DENN"],axis=1)
rowNames = ['DEPT','CLDC','DEN','DT35','FEFE','RILM','RILD']
mainlogcondensed = mainlog[rowNames]

print (mainlogcondensed)

              DEPT             CLDC                  DEN  \
0                M               IN                 G/C3   
1     Logged depth  Density Caliper  Compensated Density   
2            5.000         -999.250             -999.250   
3            5.100         -999.250             -999.250   
4            5.200         -999.250             -999.250   
5            5.300         -999.250             -999.250   
6            5.400         -999.250             -999.250   
7            5.500         -999.250             -999.250   
8            5.600         -999.250             -999.250   
9            5.700         -999.250             -999.250   
10           5.800         -999.250             -999.250   
11           5.900         -999.250             -999.250   
12           6.000         -999.250             -999.250   
13           6.100         -999.250             -999.250   
14           6.200         -999.250             -999.250   
15           6.300         -999.250     

In [4]:
mainlogtestcondensed = mainlogtest[rowNames]
print (mainlogtestcondensed)

          DEPT   CLDC    DEN   DT35   FEFE      RILM      RILD
0            m     in  g/cm3  us/ft  ohm.m     ohm.m     ohm.m
1            2  -9999  -9999  -9999  -9999     -9999     -9999
2          2.1  -9999  -9999  -9999  -9999     -9999     -9999
3          2.2  -9999  -9999  -9999  -9999     -9999     -9999
4          2.3  -9999  -9999  -9999  -9999     -9999     -9999
5          2.4  -9999  -9999  -9999  -9999     -9999     -9999
6          2.5  -9999  -9999  -9999  -9999     -9999     -9999
7          2.6  -9999  -9999  -9999  -9999     -9999     -9999
8          2.7  -9999  -9999  -9999  -9999     -9999     -9999
9          2.8  -9999  -9999  -9999  -9999     -9999     -9999
10         2.9  -9999  -9999  -9999  -9999     -9999     -9999
11           3  -9999  -9999  -9999  -9999     -9999     -9999
12         3.1  -9999  -9999  -9999  -9999     -9999     -9999
13         3.2  -9999  -9999  -9999  -9999     -9999     -9999
14         3.3  -9999  -9999  -9999  -9999     -9999   

In [5]:
for row in rowNames:
    indexNames = mainlogcondensed[ mainlogcondensed[row] == '-999.250' ].index
 
    # Delete these row indexes from dataFrame
    mainlogcondensed.drop(indexNames , inplace=True)

/home/poc/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [6]:
print(mainlogcondensed)

              DEPT             CLDC                  DEN  \
0                M               IN                 G/C3   
1     Logged depth  Density Caliper  Compensated Density   
602         65.000            8.866                1.969   
603         65.100            8.866                1.983   
604         65.200            8.866                1.999   
605         65.300            8.866                2.016   
606         65.400            8.866                2.027   
607         65.500            8.866                2.038   
608         65.600            8.866                2.047   
609         65.700            8.866                2.052   
610         65.800            8.866                2.050   
611         65.900            8.865                2.042   
612         66.000            8.865                2.032   
613         66.100            8.864                2.022   
614         66.200            8.864                2.019   
615         66.300            8.864     

In [7]:
for row in rowNames:
    indexNames = mainlogtestcondensed[ mainlogtestcondensed[row] == '-999.250' ].index
 
    # Delete these row indexes from dataFrame
    mainlogtestcondensed.drop(indexNames , inplace=True)

/home/poc/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
print(mainlogtestcondensed)

          DEPT   CLDC    DEN   DT35   FEFE      RILM      RILD
0            m     in  g/cm3  us/ft  ohm.m     ohm.m     ohm.m
1            2  -9999  -9999  -9999  -9999     -9999     -9999
2          2.1  -9999  -9999  -9999  -9999     -9999     -9999
3          2.2  -9999  -9999  -9999  -9999     -9999     -9999
4          2.3  -9999  -9999  -9999  -9999     -9999     -9999
5          2.4  -9999  -9999  -9999  -9999     -9999     -9999
6          2.5  -9999  -9999  -9999  -9999     -9999     -9999
7          2.6  -9999  -9999  -9999  -9999     -9999     -9999
8          2.7  -9999  -9999  -9999  -9999     -9999     -9999
9          2.8  -9999  -9999  -9999  -9999     -9999     -9999
10         2.9  -9999  -9999  -9999  -9999     -9999     -9999
11           3  -9999  -9999  -9999  -9999     -9999     -9999
12         3.1  -9999  -9999  -9999  -9999     -9999     -9999
13         3.2  -9999  -9999  -9999  -9999     -9999     -9999
14         3.3  -9999  -9999  -9999  -9999     -9999   

In [9]:
for row in rowNames:
    indexNames = mainlogtestcondensed[ mainlogtestcondensed[row] == '-9999' ].index
 
    # Delete these row indexes from dataFrame
    mainlogtestcondensed.drop(indexNames , inplace=True)

/home/poc/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
print(mainlogtestcondensed)

          DEPT      CLDC       DEN      DT35      FEFE      RILM       RILD
0            m        in     g/cm3     us/ft     ohm.m     ohm.m      ohm.m
158       17.7  9.023964  1.927394  68.11917  6.468544      2e+7  0.3566769
159       17.8    9.0259  1.931709  68.20406  6.452255      2e+7  0.3564982
160       17.9  9.029049  1.931903  68.19576  6.348849      2e+7  0.3572716
161         18  9.031277  1.928273  68.12599   6.09034      2e+7  0.3577394
162       18.1  9.032492  1.924739  68.09496  5.798159      2e+7  0.3571668
163       18.2  9.032492  1.921604  68.10149  5.509517      2e+7  0.3573363
164       18.3  9.032492  1.920239  68.13779  5.143405      2e+7  0.3560432
165       18.4  9.032492  1.919398  68.20276  4.540271      2e+7  0.3524973
166       18.5  9.033066  1.915225    68.291  4.008946      2e+7   0.347935
167       18.6  9.034115  1.910371  68.39084  3.820154      2e+7  0.3448997
168       18.7  9.035164  1.905204  68.47238  3.990475      2e+7  0.3426319
169       18

In [17]:
x_train = mainlogcondensed
x_test = mainlogtestcondensed

In [18]:
print(x_train.shape)
print(x_test.shape)

(7161, 7)
(8979, 7)


In [19]:
n_clusters = 5

In [20]:
x_train = x_train.drop(x_train.index[[0, 1]])

In [21]:
x_test = x_test.drop(x_test.index[[0, 1]])

In [22]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(x_train)

In [28]:
y = kmeans.predict(x_test)
print(y)

[2 2 2 ... 1 1 1]


In [31]:
plt.scatter(x_test[:, 0], x_test[:, 1], c=y, s=50, cmap='viridis')

centers = kmeans.cluster_centers
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

TypeError: unhashable type: 'slice'